### Saving and loading a SparkML model


#### Objectives:

 - Create a simple Linear Regression Model
 - Save the SparkML model
 - Load the SparkML model
 - Make predictions using the loaded SparkML model


#### Import libraries


In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

#### Creating the spark session and context


In [2]:
# Creating a spark context class
sc = SparkContext()

# Creating a spark session
spark = SparkSession \
    .builder \
    .appName("Saving and Loading a SparkML Model").getOrCreate()

In [3]:
print(spark.conf.get("spark.master"))

local[*]


#### Importing Spark ML libraries


In [3]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

#### Create a DataFrame with sample data


In [4]:
# Create a simple data set of infant height(cms) weight(kgs) chart.

mydata = [[46,2.5],[51,3.4],[54,4.4],[57,5.1],[60,5.6],[61,6.1],[63,6.4]]
  
# Mention column names of dataframe
columns = ["height", "weight"]
  
# creating a dataframe
mydf = spark.createDataFrame(mydata, columns)
  
# show data frame
mydf.show()

+------+------+
|height|weight|
+------+------+
|    46|   2.5|
|    51|   3.4|
|    54|   4.4|
|    57|   5.1|
|    60|   5.6|
|    61|   6.1|
|    63|   6.4|
+------+------+



#### Converting data frame columns into feature vectors

In this task we use the `VectorAssembler()` function to convert the dataframe columns into feature vectors. 
For our example, we use the horsepower ("hp) and weight of the car as input features and the miles-per-gallon ("mpg") as target labels.


In [5]:
assembler = VectorAssembler(
    inputCols=["height"],
    outputCol="features")

data = assembler.transform(mydf).select('features','weight')

In [6]:
data.show()

+--------+------+
|features|weight|
+--------+------+
|  [46.0]|   2.5|
|  [51.0]|   3.4|
|  [54.0]|   4.4|
|  [57.0]|   5.1|
|  [60.0]|   5.6|
|  [61.0]|   6.1|
|  [63.0]|   6.4|
+--------+------+



#### Create and Train model

We can create the model using the `LinearRegression()` class and train using the `fit()` function. 


In [7]:
# Create a LR model
lr = LinearRegression(featuresCol='features', labelCol='weight', maxIter=100)
lr.setRegParam(0.1)
# Fit the model
lrModel = lr.fit(data)

#### Save the model


In [8]:
lrModel.write().overwrite().save('/models/infantheight2.model')

#### Load the model


In [9]:
# You need LinearRegressionModel to load the model
from pyspark.ml.regression import LinearRegressionModel

In [10]:
model = LinearRegressionModel.load('/models/infantheight2.model')

#### Make Prediction


#### Predict the weight of an infant whose height is 70 CMs.


In [11]:
# This function converts a scalar number into a dataframe that can be used by the model to predict.
def predict(height):
    assembler = VectorAssembler(inputCols=["height"], outputCol="features")  # Adjusted input column name
    data = [[height, 0]]  # Changed input to reflect height
    columns = ["height", "weight"]  # Updated column names for clarity
    df = spark.createDataFrame(data, columns)
    transformed_df = assembler.transform(df).select('features', 'weight')  # Updated column selection
    predictions = model.transform(transformed_df)
    predictions.select('prediction').show()


In [12]:
predict(70)

+-----------------+
|       prediction|
+-----------------+
|7.863454719775881|
+-----------------+



### Practice exercises


#### Save the model as `babyweightprediction.model`


In [13]:
lrModel.save('/models/babyweightprediction.model')

Double-click __here__ for the solution.

<!-- Hint:

lrModel.save('babyweightprediction.model')
-->


#### Load the model `babyweightprediction.model`


In [15]:
model = LinearRegressionModel.load('/models/babyweightprediction.model')

Double-click __here__ for the solution.

<!-- Hint:

model = LinearRegressionModel.load('babyweightprediction.model')
-->


#### Predict the weight of an infant whose height is 50 CMs.


In [16]:
predict(50)

+-----------------+
|       prediction|
+-----------------+
|3.466682671116457|
+-----------------+



Double-click __here__ for the solution.

<!-- Hint:

predict(50)
-->
